In [3]:
# Imports the Google Cloud client library
from google.cloud import language_v1

# Instantiates a client
client = language_v1.LanguageServiceClient()

# The text to analyze
text = '''GENEVA – The remains of a climber discovered in the Swiss Alps in 2022 have been identified as \
    those of a British mountaineer who went missing 52 years ago, local police said on Thursday.\
        It is the latest in a series of discoveries of remains of long-missing climbers revealed as the Alps’ glaciers melt and recede because of global warming.The climber was reported missing in July 1971, \
            but search teams at the time turned up nothing, said police in the canton of Valais, south-west Switzerland.Then on Aug 22, 2022, two climbers found human remains on the Chessjengletscher glacier near Saas-Fee,\
    an Alpine village in the Saas Valley.It took a year to identify the person as experts worked their \
        way through the case files of missing climbers.Finally, with the help of Interpol Manchester and the police in Scotland, a relative was found and a DNA sample allowed them to identify the British mountaineer\
            , \the Swiss police said in a statement.\
        The climber was formally identified on Aug 30.Increasing numbers of human remains, some of them of climbers missing for decades, have been discovered in recent years as glaciers in the Alps melt because of global warming.In late July, \
            the remains of a German climber who went missing in 1986 were discovered on another Swiss glacier. AFP                  ",
'''
document = language_v1.types.Document(
    content=text, type_=language_v1.types.Document.Type.PLAIN_TEXT
)

# Detects the sentiment of the text
entity = client.analyze_entities(
    request={"document": document}
)

print(f"Text: {text}")
print(entity.entities)

Text: GENEVA – The remains of a climber discovered in the Swiss Alps in 2022 have been identified as     those of a British mountaineer who went missing 52 years ago, local police said on Thursday.        It is the latest in a series of discoveries of remains of long-missing climbers revealed as the Alps’ glaciers melt and recede because of global warming.The climber was reported missing in July 1971,             but search teams at the time turned up nothing, said police in the canton of Valais, south-west Switzerland.Then on Aug 22, 2022, two climbers found human remains on the Chessjengletscher glacier near Saas-Fee,    an Alpine village in the Saas Valley.It took a year to identify the person as experts worked their         way through the case files of missing climbers.Finally, with the help of Interpol Manchester and the police in Scotland, a relative was found and a DNA sample allowed them to identify the British mountaineer            , 	he Swiss police said in a statement.    

In [4]:
type(entity)

google.cloud.language_v1.types.language_service.AnalyzeEntitiesResponse

In [8]:
entity.entities

[name: "climber"
type_: PERSON
salience: 0.301220328
mentions {
  text {
    content: "climber"
    begin_offset: -1
  }
  type_: COMMON
}
mentions {
  text {
    content: "climber"
    begin_offset: -1
  }
  type_: COMMON
}
mentions {
  text {
    content: "climber"
    begin_offset: -1
  }
  type_: COMMON
}
, name: "mountaineer"
type_: PERSON
salience: 0.166266277
mentions {
  text {
    content: "mountaineer"
    begin_offset: -1
  }
  type_: COMMON
}
, name: "remains"
type_: OTHER
salience: 0.0640264302
mentions {
  text {
    content: "remains"
    begin_offset: -1
  }
  type_: COMMON
}
, name: "Swiss Alps"
type_: LOCATION
metadata {
  key: "wikipedia_url"
  value: "https://en.wikipedia.org/wiki/Switzerland"
}
metadata {
  key: "mid"
  value: "/m/06mzp"
}
salience: 0.0551602
mentions {
  text {
    content: "Swiss Alps"
    begin_offset: -1
  }
  type_: PROPER
}
mentions {
  text {
    content: "Swiss"
    begin_offset: -1
  }
  type_: PROPER
}
mentions {
  text {
    content: "Sw

In [12]:
for i in range(len(entity.entities)):
    print(entity.entities[i])
    if i >1:
        break
    

name: "climber"
type_: PERSON
salience: 0.301220328
mentions {
  text {
    content: "climber"
    begin_offset: -1
  }
  type_: COMMON
}
mentions {
  text {
    content: "climber"
    begin_offset: -1
  }
  type_: COMMON
}
mentions {
  text {
    content: "climber"
    begin_offset: -1
  }
  type_: COMMON
}

name: "mountaineer"
type_: PERSON
salience: 0.166266277
mentions {
  text {
    content: "mountaineer"
    begin_offset: -1
  }
  type_: COMMON
}

name: "remains"
type_: OTHER
salience: 0.0640264302
mentions {
  text {
    content: "remains"
    begin_offset: -1
  }
  type_: COMMON
}



In [13]:
entity.entities[2]

name: "remains"
type_: OTHER
salience: 0.0640264302
mentions {
  text {
    content: "remains"
    begin_offset: -1
  }
  type_: COMMON
}

In [16]:
entity.entities[3].type_

<Type.LOCATION: 2>

In [1]:
test update commit 


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1406566714.py, line 1)